# Scrape SSCASN Data 2024

In [ ]:
import requests
import warnings
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from concurrent.futures import ThreadPoolExecutor
from pymongo import MongoClient
warnings.filterwarnings('ignore')
mongo_client = None

## Connect to Database

Connect to local mongodb database

In [ ]:
def get_mongo_client():
    global mongo_client
    if mongo_client is None:
        mongo_client = MongoClient("mongodb://localhost:27017/")
    return mongo_client

def get_database():
    client = get_mongo_client()
    return client["casn2024"]
def save_to_mongo(data_batch, collection_name):
    try:
        db = get_database()
        collection = db[collection_name]
        collection.insert_many(data_batch, ordered=False)
        print(f"Batch saved successfully, {len(data_batch)} records.")
    except Exception as e:
        print(f"Failed to save data: {e}")
        return None

## Get And Save Pendidikan Data 2024

Get all pendidikan data from sscasn 2024 and save it to local mongodb database, the pendidikan_list is from the sscasn website
```json
{
  "cepat_kode": 3004000,
  "nama": "PERSAMAAN SLTA (PAKET C)",
  "tingkat_pendidikan_id": 15
}
```

In [ ]:
def get_pendidikan(tingkat):
    url = 'https://api-sscasn.bkn.go.id/2024/referensi/pendidikan'
    params = {
        'tingkat': tingkat,
        'nama': 'null',
        'limit': 2500
    }

    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-encoding': 'gzip, deflate, br, zstd',
        'accept-language': 'en-US,en;q=0.9,id;q=0.8',
        'connection': 'keep-alive',
        'host': 'api-sscasn.bkn.go.id',
        'origin': 'https://sscasn.bkn.go.id',
        'referer': 'https://sscasn.bkn.go.id/',
        'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        pendidikanData = data['data']['data']
        return pendidikanData
    else:
        print(f"Failed to retrieve data: {response.status_code}")


In [ ]:
def save_pendidikan(data):
    try:
        db = get_database()
        collection = db["pendidikan"]
        collection.insert_many(data, ordered=False)
    except Exception as e:
        print(f"Failed to save data: {e}")

In [ ]:
pendidikan_list = [{"id":"05","nama":"SD"},{"id":"10","nama":"SLTP"},{"id":"15","nama":"SLTA"},{"id":"17","nama":"SMK/SLTA Kejuruan"},{"id":"18","nama":"SLTA Keguruan"},{"id":"20","nama":"Diploma I"},{"id":"25","nama":"Diploma II"},{"id":"30","nama":"Diploma III/Sarjana Muda"},{"id":"35","nama":"Diploma IV"},{"id":"40","nama":"S-1/Sarjana"},{"id":"45","nama":"S-2"},{"id":"50","nama":"S-3/Doktor"}]

In [ ]:
for pendidikan in pendidikan_list:
    pendidikan_request = get_pendidikan(pendidikan['id'])
    save_pendidikan(pendidikan_request)

## Get And Save Formasi By Pendidikan Data 2024

Recursively get all formasi by pendidikan data from sscasn 2024 and save it to local mongodb database. Recursive function is used to get all formasi by pendidikan data because the formasi data is paginated. To store in mongodb, make sure formasi_id is unique so no duplicate data is stored.
```json
{
  "formasi_id": "54732d49-9c3c-4441-8754-789190e90017",
  "ins_nm": "Pemerintah Kota Palopo",
  "jp_nama": "CPNS",
  "formasi_nm": "UMUM",
  "jabatan_nm": "PRANATA LABORATORIUM KESEHATAN AHLI PERTAMA",
  "lokasi_nm": "Pemerintah Kota Palopo | Sekretariat Daerah | Dinas Kesehatan | UPT Laboratorium Kesehatan Daerah",
  "jumlah_formasi": 2,
  "disable": 1,
  "gaji_min": "2785700",
  "gaji_max": "4313870",
  "jumlah_ms": 0
}
```

In [ ]:
def get_data(iteration, kode_pendidikan, offset):
    try:
        data = []
        url = 'https://api-sscasn.bkn.go.id/2024/portal/spf'
        params = {
            'kode_ref_pend': kode_pendidikan,
            'pengadaan_kd': '2',
            'offset': offset,
        }
        response = requests.get(url, headers={'origin': 'https://sscasn.bkn.go.id'}, params=params)

        total_data_in_page = response.json()['data']['page']['total']
        if total_data_in_page == 10:
            offset += 10
            getData = get_data(iteration, kode_pendidikan, offset)
            if getData:
                data.extend(getData)  

        dataFormasi = response.json()['data']['data']
        for i in dataFormasi:
            data.append(i)
        return data
    except Exception as e:
        return None

In [ ]:
def get_pendidikan_from_db():
    db = get_database()
    collection = db['pendidikan']
    pendidikan = collection.find()
    return pendidikan


In [ ]:
df_pendidikan = pd.DataFrame(get_pendidikan_from_db())
kode_pend_list = df_pendidikan['kode_pend'].tolist()
batch_size = 1000

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for i, kode_pendidikan in enumerate(kode_pend_list):
        futures.append(executor.submit(get_data, i + 1, str(kode_pendidikan), 0))
    
    batch_data = []
    for future in as_completed(futures):
        data = future.result()
        if data:
            batch_data.extend(data)
            if len(batch_data) >= batch_size:
                save_to_mongo(batch_data, 'formasi')
                batch_data = []

    if batch_data:
        save_to_mongo(batch_data, 'formasi')

## Get And Save Detail Formasi Data 2024

After getting all formasi data, get the detail formasi data from sscasn 2024 and save it to local mongodb database.
```json
{
  "formasi_id": "724cb29f-9000-447e-bf55-fe253e9c6d21",
  "mpi_id": "f14769953f5111efa54d0050568fed0f",
  "instansi_id": "A5EB03E23D46F6A0E040640A040252AD",
  "ins_kd": "3021",
  "ins_nm": "Kementerian Pekerjaan Umum dan Perumahan Rakyat",
  "pengadaan_kd": "2",
  "jp_nama": "CPNS",
  "formasi_nm": "UMUM",
  "jabatan_kd": "JF0087286",
  "jabatan_nm": "PENATA KELOLA PERUMAHAN AHLI PERTAMA",
  "lokasi_nm": "KEMENTERIAN PEKERJAAN UMUM DAN PERUMAHAN RAKYAT | Direktorat Jenderal Perumahan | Balai Pelaksana Penyediaan Perumahan Kalimantan I | Subbagian Umum dan Tata Usaha",
  "pendidikan_nm": "S-1 TEKNIK INFORMATIKA / S-1 TEKNIK ARSITEKTUR / S-1 EKONOMI / S-1 MANAJEMEN / S-1 TEKNIK SIPIL / S-1 ARSITEKTUR / S-1 DESAIN INTERIOR / S-1 HUKUM / S-1 ADMINISTRASI BISNIS / S-1 TEKNIK PLANOLOGI / S-1 EKONOMI PEMBANGUNAN / S-1 TEKNOLOGI INFORMASI / S-1 PLANOLOGI / S-1 ARSITEKTUR LANSKAP / S-1 ADMINISTRASI PUBLIK / S-1 ILMU PEMERINTAHAN / S-1 EKONOMI, KEUANGAN, DAN PERBANKAN / S-1 TEKNIK PERENCANAAN WILAYAH DAN KOTA / S-1 PERENCANAAN WILAYAH DAN KOTA",
  "jumlah_formasi": 3,
  "DISABLE": 1,
  "group_tk_pendidikan_id": "",
  "kode_ref_pend": "5101087 / 5101345 / 5101775 / 5102286 / 5109100 / 5109751 / 5119036 / 5123000 / 5157011 / 5190033 / 5190061 / 5190068 / 5191078 / 5193464 / 5210315 / 5210338 / 5102812 / 5109167 / 5109169",
  "job_desc": "1. mengidentifikasi kebutuhan pengaturan bidang Penatakelolaan Perumahan tingkat kabupaten/kota;",
  "keahlian": "Mengoperasikan komputer",
  "gaji_min": "9400000",
  "gaji_max": "10000000",
  "link_web_instansi": "https://pu.go.id/pengumuman",
  "call_center_instansi": "Telepon 0818-0312-1945 / 0819-0312-1945 / Email : seleksi.asn@pu.go.id",
  "medsos_instansi": "https://www.instagram.com/kemenpupr?igsh=MWttazB6ZTNibGx4dg==",
  "helpdesk_instansi": "seleksi.asn@pu.go.id",
  "syarat_admin": [
    {
      "syarat": "Sertifikat tes kemampuan Bahasa Inggris Asli yang diterbitkan paling lama tahun 2022, dari : \r\na. ETS (Educational Testing Service) berupa : TOEFL ITP(PBT) skor minimal 450/TOEFL CBT skor minimal 131/TOEFL IBT skor minimal 45/ TOEIC skor minimal 440; atau\r\nb. IELTS skor minimal 5; atau\r\nc. Sertifikasi dengan standar penilaian lain seperti CEFR ,EF SET, English Proficiency Test, TOEFL Prediction yang diterbitkan PTN/PTS/Lembaga Kursus Swasta dengan skor setara dengan skor pada poin a dan b.",
      "is_mandatory": "\u0001"
    },
    {
      "syarat": "Pas Foto terbaru, sekurang-kurangnya menggunakan kemeja dengan warna latar berwarna merah",
      "is_mandatory": "\u0001"
    },
    {
      "syarat": "KTP asli/Surat Keterangan asli Pengganti KTP dari Dukcapil/Surat Keterangan asli Perekaman Data E-KTP",
      "is_mandatory": "\u0001"
    },
    {
      "syarat": "Surat Pernyataan Asli yang telah ditandatangani serta dibubuhi e-materai Rp. 10.000,-  sesuai format pada pengumumum Seleksi Pengadaan CPNS Kementerian PUPR Tahun 2024",
      "is_mandatory": "\u0001"
    },
    {
      "syarat": "Surat Lamaran asli yang ditujukan kepada Menteri PUPR dan sudah ditandatangani dengan tinta hitam oleh pelamar serta dibubuhi e-materai Rp. 10.000 sesuai format pada pengumumum Seleksi Pengadaan CPNS Kementerian PUPR Tahun 2024",
      "is_mandatory": "\u0001"
    },
    {
      "syarat": "Ijazah asli sesuai yang dipersyaratkan, \r\n1) Khusus bagi pelamar lulusan Perguruan Tinggi luar negeri wajib menyertakan hasil pindai (scan) Surat Penyetaraan Ijazah asli;\r\n2) Khusus bagi pelamar kebutuhan jabatan Psikolog Klinis Ahli Pertama, wajib menyertakan hasil pindai (scan) ijazah asli Sarjana (S-1) dan Profesi.",
      "is_mandatory": "\u0001"
    },
    {
      "syarat": "Transkrip/daftar nilai asli, \r\n1)Khusus bagi lulusan Perguruan Tinggi luar negeri yang melamar kebutuhan khusus Putra/Putri Lulusan Terbaik (Cum Laude) menyertakan hasil pindai (scan) surat keterangan yang menyatakan predikat kelulusannya setara “Dengan Pujian”/Cumlaude dari Kemendikburistek.\r\n2) khusus bagi pelamar kebutuhan jabatan Psikolog Klinis Ahli Pertama, wajib menyertakan hasil pindai (scan) transkrip nilai asli Sarjana (S-1) dan Profesi.",
      "is_mandatory": "\u0001"
    },
    {
      "syarat": "Sertifikat akreditasi program studi dan/atau perguruan tinggi atau hasil tangkapan layar (screenshot) pangkalan data pendidikan tinggi (PDDIKTI) atau pangkalan data Badan Akreditasi Nasional Perguruan Tinggi (BAN-PT) yang menunjukkan akreditasi program studi dan/atau perguruan tinggi pada saat kelulusan (sesuai tanggal kelulusan pada ijazah) yang sesuai dengan syarat program studi pada jabatan yang dilamar.",
      "is_mandatory": "\u0001"
    }
  ],
  "update_time": "2024-11-05 06:00:00",
  "jenis_formasi_id": "1",
  "jumlah_ms": 3
}
``` 

In [ ]:
def get_data_formasi(iteration, formasi_id):
    try:
        url = 'https://api-sscasn.bkn.go.id/2024/portal/spf/' + str(formasi_id)
        response = requests.get(url, headers={'origin': 'https://sscasn.bkn.go.id'})
        dataFormasi = response.json()['data']
        return dataFormasi
    except Exception as e:
        print('Error: ', e, formasi_id)
        return None

In [ ]:
def getIdFormasi():
    db = get_database()
    collection = db['formasi_id']
    formasi = collection.find()
    return formasi

In [ ]:
id_formasi = pd.DataFrame(getIdFormasi())
id_formasi_list = id_formasi['formasi_id'].tolist()
batch_size = 1000

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = []
    for i, formasi_id in enumerate(id_formasi_list):
        futures.append(executor.submit(get_data_formasi, i + 1, str(formasi_id)))
    
    batch_data = []
    for future in as_completed(futures):
        data = future.result()
        if data:
            batch_data.append(data)
            if len(batch_data) >= batch_size:
                save_to_mongo(batch_data, 'formasi_detail')
                batch_data = []

    if batch_data:
        save_to_mongo(batch_data, 'formasi_detail')